Connected to Python 3.10.4

In [64]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [65]:
def get_table_by_time(start_date,end_date):
    
    df =pd.read_csv(str(start_date.year -2000) +'-'+str(end_date.year - 2000)+'pl.csv')
    try :
        df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
    
    except :
        df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%y')
        
    
    table = pd.DataFrame(columns=["Club","PlayedMatches","Point","Win","Draw","Loose","GDifference","Position","GDRank"])
    
    teams = (pd.concat([df["HomeTeam"],df["AwayTeam"]])).unique()
    
    for team in teams:
        home_matches = df[df['HomeTeam'] == team]
        away_matches = df[df['AwayTeam'] == team]
        home_matches = home_matches[home_matches['Date']<end_date]
        home_matches = home_matches[home_matches['Date'] >= start_date]
        away_matches = away_matches[away_matches['Date']<end_date]
        away_matches = away_matches[away_matches['Date'] >= start_date]
        

        played_matches = len(home_matches) + len(away_matches)
        wins = len(home_matches[home_matches["FTHG"]>home_matches["FTAG"]]) + len(away_matches[away_matches['FTAG'] > away_matches['FTHG']])
        draws = len(home_matches[home_matches['FTHG'] == home_matches['FTAG']]) + len(away_matches[away_matches['FTHG'] == away_matches['FTAG']])
        looses = played_matches - wins - draws
        point = wins*3 + draws

        goal_difference = sum(home_matches['FTHG'] - home_matches['FTAG']) + sum(away_matches['FTAG'] - away_matches['FTHG'])

        data_to_add = {
            'Club': team,
            'PlayedMatches': played_matches,
            'Point': point,
            'Win': wins,
            'Draw': draws,
            'Lose': looses,
            'GDifference': goal_difference,
            'Position': 0,
            'GDRank': 0,
        }
        
        new_row = pd.DataFrame([data_to_add])
        table = pd.concat([table, new_row], ignore_index=True)
        
    table = table.sort_values(by="GDifference", ascending=False)
    table['GDRank'] = range(1,len(table)+1)
    
    table = table.sort_values(by="Point", ascending=False)
    table['Position'] = range(1,len(table)+1)
    
    table = table[['Club','Position','GDRank']]
    return table.reset_index(drop=True)

In [66]:
mid_season_table = pd.DataFrame()
end_of_season_table = pd.DataFrame()

start_t =  2022
end_t = 2023

for i in range (10):
    tmp = get_table_by_time(pd.to_datetime(str(start_t) + '-08-01'),pd.to_datetime(str(end_t) + '-01-05'))
    mid_season_table = pd.concat([mid_season_table,tmp])
    
    tmp = get_table_by_time(pd.to_datetime(str(start_t) + '-08-01'),pd.to_datetime(str(end_t) + '-07-01'))
    end_of_season_table = pd.concat([end_of_season_table,tmp])
    
    end_t = start_t
    start_t -=1
    

In [67]:
mid_season_table.head(30)

,Club,Position,GDRank
0,Arsenal,1,2
1,Man City,2,1
2,Newcastle,3,3
3,Man United,4,6
4,Tottenham,5,5
5,Liverpool,6,4
6,Fulham,7,8
7,Brighton,8,7
8,Brentford,9,10
9,Chelsea,10,9


In [68]:
end_of_season_table.head(30)

,Club,Position,GDRank
0,Man City,1,1
1,Arsenal,2,2
2,Man United,3,6
3,Newcastle,4,3
4,Liverpool,5,4
5,Brighton,6,5
6,Aston Villa,7,9
7,Tottenham,8,8
8,Brentford,9,7
9,Fulham,10,10


In [69]:
combined_table = pd.merge(mid_season_table, end_of_season_table, on='Club', suffixes=('_mid', '_end'))
combined_table['AbsDiff_Position'] = abs(combined_table['Position_end'] - combined_table['Position_mid'])
combined_table['AbsDiff_GDRank'] = abs(combined_table['Position_end'] - combined_table['GDRank_mid'])
combined_table = combined_table.sort_values(by='Position_end').reset_index(drop=True)

In [70]:
combined_table

,Club,Position_mid,GDRank_mid,Position_end,GDRank_end,AbsDiff_Position,AbsDiff_GDRank
0,Man City,2,1,1,1,1,0
1,Man City,1,1,1,1,0,0
2,Man City,1,1,1,1,0,0
3,Man City,2,1,1,1,1,0
4,Man City,3,1,1,1,2,0
...,...,...,...,...,...,...,...
1560,West Brom,13,12,20,17,7,8
1561,Southampton,9,8,20,20,11,12
1562,West Brom,17,16,20,17,3,4
1563,Huddersfield,20,19,20,20,0,1


In [71]:
print("abs pos dif:  ",combined_table["AbsDiff_Position"].sum(),"  abs gd dif:  ",combined_table["AbsDiff_GDRank"].sum())

abs pos dif:   5919   abs gd dif:   5738


In [72]:

for i in range(2,22,2 ):
    top_pos = combined_table[combined_table['Position_end'] < i]
    posdif = top_pos["AbsDiff_Position"].sum()
    gdrdif = top_pos["AbsDiff_GDRank"].sum()
    print("top: ", i, " pos dif: ",posdif," gdr dif: ", gdrdif," gd/pd: ",gdrdif/posdif)

top:  2  pos dif:  267  gdr dif:  231  gd/pd:  0.8651685393258427
top:  4  pos dif:  794  gdr dif:  756  gd/pd:  0.9521410579345088
top:  6  pos dif:  1366  gdr dif:  1300  gd/pd:  0.9516837481698389
top:  8  pos dif:  2069  gdr dif:  1986  gd/pd:  0.9598840019333011
top:  10  pos dif:  2721  gdr dif:  2633  gd/pd:  0.9676589489158398
top:  12  pos dif:  3445  gdr dif:  3319  gd/pd:  0.9634252539912918
top:  14  pos dif:  4085  gdr dif:  3931  gd/pd:  0.9623011015911873
top:  16  pos dif:  4670  gdr dif:  4494  gd/pd:  0.9623126338329765
top:  18  pos dif:  5223  gdr dif:  5058  gd/pd:  0.9684089603676048
top:  20  pos dif:  5701  gdr dif:  5522  gd/pd:  0.9686019996491844
